In [1]:
import numpy as np
from scipy import sparse
import xgboost as xgb
import pandas as pd
import re
import string
import time

from sklearn import preprocessing, pipeline, metrics, model_selection
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import feature_selection
from itertools import product

import osgeo 
import fiona
import json
import geopandas as gpd
import geocoder
from scipy import sparse
from shapely.geometry import Point
from geopandas.tools import sjoin
from geopy.distance import vincenty

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline 

C:\Users\Eric Yang\Anaconda2\envs\snakes\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train_data = pd.read_json(r'C:\Users\Eric Yang\Desktop\Kaggle Comptetion\Current Competition\train.json')

test_data = pd.read_json(r'C:\Users\Eric Yang\Desktop\Kaggle Comptetion\Current Competition\test.json') 

In [3]:
import random
index=list(range(train_data.shape[0]))
random.shuffle(index)
a=[np.nan]*len(train_data)
b=[np.nan]*len(train_data)
c=[np.nan]*len(train_data)

for i in range(5):
    building_level={}
    for j in train_data['manager_id'].values:
        building_level[j]=[0,0,0]
    test_index=index[int((i*train_data.shape[0])/5):int(((i+1)*train_data.shape[0])/5)]
    train_index=list(set(index).difference(test_index))
    for j in train_index:
        temp=train_data.iloc[j]
        if temp['interest_level']=='low':
            building_level[temp['manager_id']][0]+=1
        if temp['interest_level']=='medium':
            building_level[temp['manager_id']][1]+=1
        if temp['interest_level']=='high':
            building_level[temp['manager_id']][2]+=1
    for j in test_index:
        temp=train_data.iloc[j]
        if sum(building_level[temp['manager_id']])!=0:
            a[j]=building_level[temp['manager_id']][0]*1.0/sum(building_level[temp['manager_id']])
            b[j]=building_level[temp['manager_id']][1]*1.0/sum(building_level[temp['manager_id']])
            c[j]=building_level[temp['manager_id']][2]*1.0/sum(building_level[temp['manager_id']])
train_data['manager_level_low']=a
train_data['manager_level_medium']=b
train_data['manager_level_high']=c



a=[]
b=[]
c=[]
building_level={}
for j in train_data['manager_id'].values:
    building_level[j]=[0,0,0]
for j in range(train_data.shape[0]):
    temp=train_data.iloc[j]
    if temp['interest_level']=='low':
        building_level[temp['manager_id']][0]+=1
    if temp['interest_level']=='medium':
        building_level[temp['manager_id']][1]+=1
    if temp['interest_level']=='high':
        building_level[temp['manager_id']][2]+=1

for i in test_data['manager_id'].values:
    if i not in building_level.keys():
        a.append(np.nan)
        b.append(np.nan)
        c.append(np.nan)
    else:
        a.append(building_level[i][0]*1.0/sum(building_level[i]))
        b.append(building_level[i][1]*1.0/sum(building_level[i]))
        c.append(building_level[i][2]*1.0/sum(building_level[i]))
test_data['manager_level_low']=a
test_data['manager_level_medium']=b
test_data['manager_level_high']=c

manager_variable = ['manager_level_low','manager_level_medium','manager_level_high']

In [4]:
train_size = train_data.shape[0]

### Create target variables

In [5]:
train_data['target'] = train_data['interest_level'].apply(lambda x: 0 if x=='low' else 1 if x=='medium' else 2)
train_data['low'] = train_data['interest_level'].apply(lambda x: 1 if x=='low' else 0)
train_data['medium'] = train_data['interest_level'].apply(lambda x: 1 if x=='medium' else 0)
train_data['high'] = train_data['interest_level'].apply(lambda x: 1 if x=='high' else 0)

## Merge training and testing data

In [6]:
full_data=pd.concat([train_data
                       ,test_data])

In [7]:
full_data.ix[4620, 'price']  = 1025
full_data.ix[12168, 'price']  = 3400
full_data.ix[32611, 'price']  = 10000

full_data.ix[51229,'latitude'] = 40.831835
full_data.ix[51229,'longitude'] = -73.921021
full_data.ix[113035, 'latitude'] = 40.869699
full_data.ix[113035,'longitude'] = -73.243134
full_data.ix[104822, 'latitude'] = 40.752816
full_data.ix[104822,'longitude'] = -73.970854
full_data.ix[72896, 'latitude'] = 40.772661
full_data.ix[72896,'longitude'] = -73.955558
full_data.ix[78568, 'latitude'] = 40.763670
full_data.ix[78568,'longitude'] = -73.958609
full_data.ix[109135, 'latitude'] = 40.778075
full_data.ix[109135,'longitude'] = -73.952238
full_data.ix[17772, 'latitude'] = 40.748368
full_data.ix[17772,'longitude'] = -73.976599
full_data.ix[67902, 'latitude'] = 40.805668
full_data.ix[67902,'longitude'] = -73.941987
full_data.ix[21168, 'latitude'] = 40.731960
full_data.ix[21168,'longitude'] = -74.002091
full_data.ix[55585, 'latitude'] = 40.737547
full_data.ix[55585,'longitude'] = -73.984084
full_data.ix[45416, 'latitude'] = 40.754539
full_data.ix[45416,'longitude'] = -73.998372
full_data.ix[17772, 'latitude'] = 40.747632
full_data.ix[17772,'longitude'] = -73.974814
full_data.ix[109135, 'latitude'] = 40.775065
full_data.ix[109135,'longitude'] = -73.948073
full_data.ix[39798, 'latitude' ] = 40.747632
full_data.ix[39798,'longitude' ] = -73.974814

In [8]:
missing_data = pd.read_csv(r'C:\Users\Eric Yang\Desktop\Kaggle Comptetion\Current Competition\a_few_missing2.csv')

In [9]:
full_data.fillna(0, inplace = True)
full_data.loc[(full_data.longitude == 0) | (full_data.latitude == 0), 'latitude'] = missing_data.lat.values
full_data.loc[(full_data.longitude == 0) | (full_data.latitude == 0), 'longitude'] = missing_data.long.values

# Geo Processing

In [ ]:
missingCoords = full_data[(full_data.longitude == 0) | (full_data.latitude == 0)]
missingGeoms = (missingCoords.street_address + ', New York').apply(geocoder.google)

full_data.loc[(full_data.longitude == 0) | (full_data.latitude == 0), 'latitude'] = missingGeoms.apply(lambda x: x.lat)
full_data.loc[(full_data.longitude == 0) | (full_data.latitude == 0), 'longitude'] = missingGeoms.apply(lambda x: x.lng)

missing_data = pd.DataFrame({'lat':missingGeoms.apply(lambda x: x.lat), 'long':missingGeoms.apply(lambda x: x.lng)})
missing_data.to_csv(r'C:\Users\Eric Yang\Desktop\Kaggle Comptetion\Current Competition\a_few_missing2.csv')

In [10]:
full_data['geometry'] = full_data.apply(lambda x: Point((float(x.longitude), float(x.latitude))), axis=1)

In [11]:
poly = gpd.GeoDataFrame.from_file(r'C:\Users\Eric Yang\Desktop\Kaggle Comptetion\Current Competition\nynta.geojson')
gdat = gpd.GeoDataFrame(full_data, crs = poly.crs, geometry='geometry')
geo_data = sjoin(gdat,poly, how='left', op='within')

In [12]:
geo_var = ['boroname','borocode','ntaname']
full_data = pd.merge(left = full_data, right = geo_data[geo_var],left_index=True,right_index=True, how = 'left')

### Group Variables

In [13]:
num_vars = ['bathrooms','bedrooms','latitude','longitude','price']
cat_vars = ['building_id','manager_id','display_address','street_address', 'borocode','ntaname']
text_vars = ['description','features']
date_var = 'created'
image_var = 'photos'
id_var = 'listing_id'

### Date variable

In [14]:
full_data['created_datetime'] = pd.to_datetime(full_data['created'], format="%Y-%m-%d %H:%M:%S")

In [15]:
# full_data['created_year']=full_data['created_datetime'].apply(lambda x:x.year) ## low variant
full_data['created_month']=full_data['created_datetime'].apply(lambda x:x.month)
full_data['created_day']=full_data['created_datetime'].apply(lambda x:x.day)
full_data['created_dayofweek']=full_data['created_datetime'].apply(lambda x:x.dayofweek)
full_data['created_dayofyear']=full_data['created_datetime'].apply(lambda x:x.dayofyear)
full_data['created_weekofyear']=full_data['created_datetime'].apply(lambda x:x.weekofyear)
full_data['created_hour']=full_data['created_datetime'].apply(lambda x:x.hour)
full_data['created_epoch']=full_data['created_datetime'].apply(lambda x:x.value//10**9)

date_num_vars = ['created_month','created_dayofweek','created_dayofyear'
                 ,'created_weekofyear','created_hour','created_epoch']

## Additional Numeric Variables

In [16]:
# full_data['price']=full_data['price'].apply(np.log)

In [17]:
full_data['rooms'] = full_data['bedrooms'] + full_data['bathrooms'] 
full_data['num_of_photos'] = full_data['photos'].apply(lambda x:len(x))
full_data['num_of_features'] = full_data['features'].apply(lambda x:len(x))
full_data['len_of_desc'] = full_data['description'].apply(lambda x:len(x))
full_data['words_of_desc'] = full_data['description'].apply(lambda x:len(re.sub('['+string.punctuation+']', '', x).split()))


full_data['nums_of_desc'] = full_data['description']\
        .apply(lambda x:re.sub('['+string.punctuation+']', '', x).split())\
        .apply(lambda x: len([s for s in x if s.isdigit()]))
        
full_data['has_phone'] = full_data['description'].apply(lambda x:re.sub('['+string.punctuation+']', '', x).split())\
        .apply(lambda x: [s for s in x if s.isdigit()])\
        .apply(lambda x: len([s for s in x if len(str(s))==10]))\
        .apply(lambda x: 1 if x>0 else 0)
full_data['has_email'] = full_data['description'].apply(lambda x: 1 if '@renthop.com' in x else 0)
full_data['price_latitue'] = (full_data["price"])/ (full_data["latitude"]+1.0) 
full_data['price_longtitude'] =  (full_data["price"])/ (full_data["longitude"]-1.0) 

additional_num_vars = ['rooms','num_of_photos','num_of_features','len_of_desc',
                    'words_of_desc','has_phone','has_email','price_latitue','price_longtitude']


### Numeric interactions

In [18]:
full_data['avg_word_len'] = full_data[['len_of_desc','words_of_desc']]\
                                    .apply(lambda x: x[0]/x[1] if x[1]!=0 else 0, axis=1)
    
full_data['price_per_room'] = full_data[['price','rooms']].apply(lambda x: x[0]/x[1] if x[1]!=0 else 0, axis=1)
full_data['price_per_bedroom'] = full_data[['price','bedrooms']].apply(lambda x: x[0]/x[1] if x[1]!=0 else 0, axis=1)
full_data['price_per_bathroom'] = full_data[['price','bathrooms']].apply(lambda x: x[0]/x[1] if x[1]!=0 else 0, axis=1)
full_data['price_per_photo'] = full_data[['price','num_of_photos']].apply(lambda x: x[0]/x[1] if x[1]!=0 else 0, axis=1)


full_data['photos_per_room'] = full_data[['num_of_photos','rooms']].apply(lambda x: x[0]/x[1] if x[1]!=0 else 0, axis=1)

interactive_num_vars = ['avg_word_len','price_per_room','price_per_bedroom','price_per_bathroom','price_per_photo',
                        'photos_per_room']

### Categorical Features

#### 1. Label Encoding

In [19]:
full_data.borocode.fillna(6, inplace = True)
full_data.ntaname.fillna('other', inplace = True)

In [20]:
LBL = preprocessing.LabelEncoder()

LE_vars=[]
LE_map=dict()
for cat_var in cat_vars:
    print ("Label Encoding %s" % (cat_var))
    LE_var=cat_var+'_le'
    full_data[LE_var]=LBL.fit_transform(full_data[cat_var])
    LE_vars.append(LE_var)
    LE_map[cat_var]=LBL.classes_
    
print ("Label-encoded feaures: %s" % (LE_vars))

Label Encoding building_id
Label Encoding manager_id
Label Encoding display_address
Label Encoding street_address
Label Encoding borocode
Label Encoding ntaname
Label-encoded feaures: ['building_id_le', 'manager_id_le', 'display_address_le', 'street_address_le', 'borocode_le', 'ntaname_le']


In [21]:
LE_vars.remove('ntaname_le')

#### 2. One Hot Encoding

In [22]:
OHE = preprocessing.OneHotEncoder(sparse=True)
start=time.time()
OHE.fit(full_data[LE_vars])
OHE_sparse=OHE.transform(full_data[LE_vars])
                                   
print ('One-hot-encoding finished in %f seconds' % (time.time()-start))


OHE_vars = [var[:-3] + '_' + str(level).replace(' ','_')\
                for var in cat_vars for level in LE_map[var] ]

print ("OHE_sparse size :" ,OHE_sparse.shape)
print ("One-hot encoded catgorical feature samples : %s" % (OHE_vars[:100]))

One-hot-encoding finished in 0.301259 seconds
OHE_sparse size : (124011, 57874)
One-hot encoded catgorical feature samples : ['building_0', 'building_00005cb939f9986300d987652c933e15', 'building_00024d77a43f0606f926e2312513845c', 'building_000ae4b7db298401cdae2b0ba1ea8146', 'building_0012f1955391bca600ec301035b97b65', 'building_0021440c04241281a436ec21accc40b1', 'building_002d1eba40aa0a6610e04ff20543585f', 'building_003d8740e21484dcc2280639b25539a4', 'building_00480e54b53fe77d17964be3f8307a99', 'building_00553d95d22484bcc36831c9248d1dbc', 'building_0055c8662ba19e95f78df97592d2b83e', 'building_0056dbdf2881b76f2a0171eb753ec9e0', 'building_0059ae562b9e338a59eaf962cb3eedd2', 'building_005e0f8d7fb7b92be351cbf1dd985149', 'building_0067f166111490e7af7f1a878a67bb5e', 'building_0070bc94a3f80aa717bb15708e98ba54', 'building_0071cda335745940cdae1dc31abfc701', 'building_0078281cd69f4bfec17e42e5cf5eecd9', 'building_0078c2ab46afba9969637ac83621901e', 'building_007ae1cd90420f18bad7b6892a9a1411', 'buil

#### 3. Leave-one-out Encoding

Based on the paper "A Preprocessing Scheme for High-Cardinality Categorical Attributes in Classification and Prediction Problems"

http://helios.mm.di.uoa.gr/~rouvas/ssi/sigkdd/sigkdd.vol3.1/barreca.ps

** A couple of Kaggle scripts: **

R version: by Braden Murray: https://www.kaggle.com/brandenkmurray/two-sigma-connect-rental-listing-inquiries/it-is-lit/comments

Python Version 1, by Stanislav Ushakov
https://www.kaggle.com/stanislavushakov/two-sigma-connect-rental-listing-inquiries/python-version-of-it-is-lit-by-branden/comments

Python Version 2, by Rakhlin
https://www.kaggle.com/rakhlin/two-sigma-connect-rental-listing-inquiries/another-python-version-of-it-is-lit-by-branden/code


In [23]:
##Create a function to encode high-cardinality cateogrical features

def designate_single_observations(df1, df2, column):
    ps = df1[column].append(df2[column])
    grouped = ps.groupby(ps).size().to_frame().rename(columns={0: "size"})
    df1.loc[df1.join(grouped, on=column, how="left")["size"] <= 1, column] = -1
    df2.loc[df2.join(grouped, on=column, how="left")["size"] <= 1, column] = -1
    return df1, df2


def hcc_encode(train_df, test_df, variable, target, prior_prob, k, f=1, g=1, r_k=None, update_df=None):
    """
    See "A Preprocessing Scheme for High-Cardinality Categorical Attributes in
    Classification and Prediction Problems" by Daniele Micci-Barreca
    """
    hcc_name = "_".join(["hcc", variable, target])

    grouped = train_df.groupby(variable)[target].agg({"size": "size", "mean": "mean"})
    grouped["lambda"] = 1 / (g + np.exp((k - grouped["size"]) / f))
    grouped[hcc_name] = grouped["lambda"] * grouped["mean"] + (1 - grouped["lambda"]) * prior_prob

    df = test_df[[variable]].join(grouped, on=variable, how="left")[hcc_name].fillna(prior_prob)
    if r_k: df *= np.random.uniform(1 - r_k, 1 + r_k, len(test_df))     # Add uniform noise. Not mentioned in original paper

    if update_df is None: update_df = test_df
    if hcc_name not in update_df.columns: update_df[hcc_name] = np.nan
    update_df.update(df)
    return

In [24]:
for col in ('building_id', 'manager_id', 'display_address'):
    train_data, test_data = designate_single_observations(train_data, test_data, col)
    
prior_low, prior_medium, prior_high = train_data[["low", "medium", "high"]].mean() 

skf = model_selection.StratifiedKFold(5)
attributes = product(("building_id", "manager_id"), zip(("medium", "high"), (prior_medium, prior_high)))
for variable, (target, prior) in attributes:
    hcc_encode(train_data, test_data, variable, target, prior, k=5, r_k=None)
    for train, test in skf.split(np.zeros(len(train_data)), train_data['interest_level']):
        hcc_encode(train_data.iloc[train], train_data.iloc[test], variable, target, prior, k=5, r_k=0.01,
                   update_df=train_data)
        
hcc_data = pd.concat([train_data[['building_id', 'manager_id', 'display_address',
            'hcc_building_id_medium','hcc_building_id_high',
            'hcc_manager_id_medium','hcc_manager_id_high']],
           test_data[['building_id', 'manager_id', 'display_address',
            'hcc_building_id_medium','hcc_building_id_high',
            'hcc_manager_id_medium','hcc_manager_id_high']]
           ]
          )
full_data['building_id'] = hcc_data['building_id']
full_data['manager_id'] = hcc_data['manager_id']
full_data['display_address'] = hcc_data['display_address']
full_data['hcc_building_id_medium'] = hcc_data['hcc_building_id_medium']
full_data['hcc_building_id_high'] = hcc_data['hcc_building_id_high']
full_data['hcc_manager_id_medium'] = hcc_data['hcc_manager_id_medium']
full_data['hcc_manager_id_high'] = hcc_data['hcc_manager_id_high']
hcc_vars = ['hcc_building_id_medium','hcc_building_id_high','hcc_manager_id_medium','hcc_manager_id_high']    

# Clustering

In [25]:
full_data.columns

Index(['bathrooms', 'bedrooms', 'building_id', 'created', 'description',
       'display_address', 'features', 'high', 'interest_level', 'latitude',
       'listing_id', 'longitude', 'low', 'manager_id', 'manager_level_high',
       'manager_level_low', 'manager_level_medium', 'medium', 'photos',
       'price', 'street_address', 'target', 'geometry', 'boroname', 'borocode',
       'ntaname', 'created_datetime', 'created_month', 'created_day',
       'created_dayofweek', 'created_dayofyear', 'created_weekofyear',
       'created_hour', 'created_epoch', 'rooms', 'num_of_photos',
       'num_of_features', 'len_of_desc', 'words_of_desc', 'nums_of_desc',
       'has_phone', 'has_email', 'price_latitue', 'price_longtitude',
       'avg_word_len', 'price_per_room', 'price_per_bedroom',
       'price_per_bathroom', 'price_per_photo', 'photos_per_room',
       'building_id_le', 'manager_id_le', 'display_address_le',
       'street_address_le', 'borocode_le', 'ntaname_le',
       'hcc_building_

In [26]:
from sklearn.cluster import DBSCAN
scale = preprocessing.StandardScaler()
cluster_vars = ['created_weekofyear','latitude', 'longitude','ntaname_le', 'bedrooms']
cluster_df = full_data[cluster_vars]
cluster_df['bedrooms'] = cluster_df['bedrooms'].clip_upper(5)
cluster_df['bedrooms'] = cluster_df['bedrooms'].map(lambda x: 0.7 * x)
cluster_df['created_weekofyear'] = cluster_df['created_weekofyear'].map(lambda x: 6 * x)
cluster_df[cluster_vars] = scale.fit_transform(cluster_df[cluster_vars])
db = DBSCAN(eps=0.28, min_samples=25).fit(cluster_df)

In [27]:
labels = db.labels_

In [28]:
label_set = set(labels)

In [29]:
labels

array([ -1,   0,   1, ...,  52,  17, 110], dtype=int64)

In [30]:
full_data['clusters'] = labels

In [33]:
(full_data[full_data['clusters'] == 1].latitude.sum(axis = 0)/len(full_data[full_data['clusters'] == 1])\
    ,full_data[full_data['clusters'] == 1].longitude.sum(axis = 0)/len(full_data[full_data['clusters'] == 1]))

(40.732916676970639, -74.001560661128295)

In [33]:
centroids = []
for i in label_set:
   centroids.append((full_data[full_data['clusters'] == i].latitude.sum(axis = 0)/len(full_data[full_data['clusters'] == i]),\
                    full_data[full_data['clusters'] == i].longitude.sum(axis = 0)/len(full_data[full_data['clusters'] == i])))
    

In [34]:
np.asarray(full_data[geo])

NameError: name 'geo' is not defined

In [ ]:
centroids = np.asarray(centroids).reshape(-1,2)

In [ ]:
geo = ['latitude', 'longitude']

In [ ]:
geo = ['latitude', 'longitude']
full_data['coords'] = list(zip(full_data.latitude, full_data.longitude))

In [ ]:
full_data['coords'].head(1)

In [ ]:
vincenty(full_data['coords'].head(1), centroids[0]).meters

In [ ]:
def distance(x):
    global centroids
    global label_set
    dist = []
    for label in label_set:
        df = full_data[full_data['coords'] == label]
        for cent in centroids:
                vincenty(x, cent).meters

In [ ]:
from scipy.spatial import distance

k_euclid_distance = [distance.cdist(np.asarray(full_data[geo]), cent, 'euclidean') for cent in centroids] #each point to each cluster center
dist = [np.min(ke,axis = 1) for ke in k_euclid_distance]

In [ ]:
full_data[full_data['clusters'] == 11].ntaname_le.value_counts()

### Text Features

#### 1. Features

In [31]:
full_data["features"].apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x]))
cntvec = CountVectorizer(stop_words='english', max_features=200)
feature_sparse =cntvec.fit_transform(full_data["features"]\
                                     .apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x])))

feature_vars = ['feature_' + v for v in cntvec.vocabulary_]

#### 2. Description

##### tf-idf not working, instead, using CountVectorizer, not really helping

In [32]:
# tfidf = TfidfVectorizer(stop_words='english', max_features=10)
# desc_sparse = tfidf.fit_transform(full_data["description"])
# desc_vars = ['desc_' + v for v in tfidf.get_feature_names()]

In [33]:
cntvec = CountVectorizer(stop_words='english', max_features=100)
desc_sparse = cntvec.fit_transform(full_data["description"])
desc_vars = ['desc_' + v for v in cntvec.vocabulary_]

##### word2vec - to be added

#### Street Address

In [34]:
# embedding

cntvec = CountVectorizer(stop_words='english', max_features=10)
st_addr_sparse = cntvec.fit_transform(full_data["street_address"])
st_addr_vars = ['desc_' + v for v in cntvec.vocabulary_]

In [35]:
full_data.columns

Index(['bathrooms', 'bedrooms', 'building_id', 'created', 'description',
       'display_address', 'features', 'high', 'interest_level', 'latitude',
       'listing_id', 'longitude', 'low', 'manager_id', 'manager_level_high',
       'manager_level_low', 'manager_level_medium', 'medium', 'photos',
       'price', 'street_address', 'target', 'geometry', 'boroname', 'borocode',
       'ntaname', 'created_datetime', 'created_month', 'created_day',
       'created_dayofweek', 'created_dayofyear', 'created_weekofyear',
       'created_hour', 'created_epoch', 'rooms', 'num_of_photos',
       'num_of_features', 'len_of_desc', 'words_of_desc', 'nums_of_desc',
       'has_phone', 'has_email', 'price_latitue', 'price_longtitude',
       'avg_word_len', 'price_per_room', 'price_per_bedroom',
       'price_per_bathroom', 'price_per_photo', 'photos_per_room',
       'building_id_le', 'manager_id_le', 'display_address_le',
       'street_address_le', 'borocode_le', 'ntaname_le',
       'hcc_building_

### Numberic vs Categorical Interactions

In [36]:
price_by_manager = full_data.groupby('manager_id')['price'].agg([np.min,np.max,np.median,np.mean]).reset_index()
price_by_manager.columns = ['manager_id','min_price_by_manager',
                            'max_price_by_manager','median_price_by_manager','mean_price_by_manager']
full_data = pd.merge(full_data,price_by_manager, how='left',on='manager_id')

created_epoch_by_manager = full_data.groupby('manager_id')['created_epoch'].agg([np.min,np.max,np.median,np.mean]).reset_index()
created_epoch_by_manager.columns = ['manager_id','min_created_epoch_by_manager',
                            'max_created_epoch_by_manager','median_created_epoch_by_manager','mean_created_epoch_by_manager']
full_data = pd.merge(full_data,created_epoch_by_manager, how='left',on='manager_id')


price_by_building = full_data.groupby('building_id')['price'].agg([np.min,np.max,np.median,np.mean]).reset_index()
price_by_building.columns = ['building_id','min_price_by_building',
                            'max_price_by_building','median_price_by_building','mean_price_by_building']
full_data = pd.merge(full_data,price_by_building, how='left',on='building_id')


created_epoch_by_building = full_data.groupby('building_id')['created_epoch'].agg([np.min,np.max,np.median,np.mean]).reset_index()
price_by_building.columns = ['building_id','min_created_epoch_by_building',
                            'max_created_epoch_by_building','median_created_epoch_by_building','mean_created_epoch_by_building']
full_data = pd.merge(full_data,price_by_building, how='left',on='building_id')

price_by_disp_addr = full_data.groupby('display_address')['price'].agg([np.min,np.max,np.median,np.mean]).reset_index()
price_by_disp_addr.columns = ['display_address','min_price_by_disp_addr',
                            'max_price_by_disp_addr','median_price_by_disp_addr','mean_price_by_disp_addr']
full_data = pd.merge(full_data,price_by_disp_addr, how='left',on='display_address')

# remove one


full_data['price_percentile_by_manager']=\
            full_data[['price','min_price_by_manager','max_price_by_manager']]\
            .apply(lambda x:(x[0]-x[1])/(x[2]-x[1]) if (x[2]-x[1])!=0 else 0.5,
                  axis=1)
full_data['price_percentile_by_building']=\
            full_data[['price','min_price_by_building','max_price_by_building']]\
            .apply(lambda x:(x[0]-x[1])/(x[2]-x[1]) if (x[2]-x[1])!=0 else 0.5,
                  axis=1)
full_data['price_percentile_by_disp_addr']=\
            full_data[['price','min_price_by_disp_addr','max_price_by_disp_addr']]\
            .apply(lambda x:(x[0]-x[1])/(x[2]-x[1]) if (x[2]-x[1])!=0 else 0.5,
                  axis=1)


full_data['created_epoch_percentile_by_manager']=\
            full_data[['created_epoch','min_created_epoch_by_manager','max_created_epoch_by_manager']]\
            .apply(lambda x:(x[0]-x[1])/(x[2]-x[1]) if (x[2]-x[1])!=0 else 0.5,
                  axis=1)


In [37]:
price_by_cluster = full_data.groupby('clusters')['price'].agg([np.min,np.max,np.median,np.mean]).reset_index()
price_by_cluster.columns = ['clusters','min_price_by_clusters',
                            'max_price_by_clusters','median_price_by_clusters','mean_price_by_clusters']
full_data = pd.merge(full_data,price_by_cluster, how='left',on='clusters')


In [38]:
full_data['price_percentile_by_clusters']=\
            full_data[['clusters','min_price_by_clusters','max_price_by_clusters']]\
            .apply(lambda x:(x[0]-x[1])/(x[2]-x[1]) if (x[2]-x[1])!=0 else 0.5,
                  axis=1)


In [39]:
created_epoch_by_display_address = full_data.groupby('display_address')['created_epoch'].agg([np.min,np.max,np.median,np.mean]).reset_index()
created_epoch_by_display_address.columns = ['display_address','min_created_epoch_by_display_address',
                            'max_created_epoch_by_display_address','median_created_epoch_by_display_address','mean_created_epoch_by_display_address']
full_data = pd.merge(full_data,created_epoch_by_display_address, how='left',on='display_address')


In [40]:
num_cat_vars = ['median_price_by_manager','mean_price_by_manager',
                'median_price_by_building','mean_price_by_building',
                'median_price_by_disp_addr','mean_price_by_disp_addr',
                'median_created_epoch_by_manager','mean_created_epoch_by_manager',
                'price_percentile_by_manager','price_percentile_by_building',
                'price_percentile_by_disp_addr','created_epoch_percentile_by_manager',
                'median_price_by_clusters', 'mean_price_by_clusters', 
                'price_percentile_by_clusters','listing_id','listing_id_pos',
                'median_created_epoch_by_display_address','mean_created_epoch_by_display_address'
               ]

### Listing ID matters

In [41]:
min_listing_id = full_data['listing_id'].min()
max_listing_id = full_data['listing_id'].max()
full_data['listing_id_pos']=full_data['listing_id'].apply(lambda x:np.float64((x-min_listing_id+1))/(max_listing_id-min_listing_id+1))
num_vars.append('listing_id')
num_vars.append('listing_id_pos')

In [42]:
full_vars = num_vars + date_num_vars + additional_num_vars + interactive_num_vars + LE_vars + hcc_vars + num_cat_vars

In [50]:
num_vars = ['bathrooms',
 'bedrooms',
 'latitude',
 'longitude',
 'price']

## Centroid

In [43]:
centroid = (full_data.latitude.sum(axis = 0)/len(full_data)\
    ,full_data.longitude.sum(axis = 0)/len(full_data))

In [44]:
from scipy.spatial import distance
geo = ['latitude', 'longitude']
full_data['coords'] = list(zip(full_data.latitude, full_data.longitude))
def dist_to_center(x):
    true_center = np.array((40.7128,-74.0059)).reshape(-1,2)
    k = distance.cdist(np.array(x).reshape(-1,2), true_center, 'euclidean') 
    return k.astype(float)
full_data['dist_to_center'] = full_data['coords'].map(dist_to_center)
full_data['dist_to_center'] = full_data['dist_to_center'].str[0]
full_data['dist_to_center'] = full_data['dist_to_center'].str[0]

In [45]:
dist_var = ['dist_to_center']

### Create training and testing data

In [51]:
%%time


##Baseline with features from "features" and street address

full_vars = num_vars + date_num_vars\
+ additional_num_vars + interactive_num_vars + LE_vars + hcc_vars + num_cat_vars + manager_variable + dist_var

train_x = sparse.hstack([full_data[full_vars],feature_sparse,st_addr_sparse]).tocsr()[:train_size]
train_y = full_data['target'][:train_size].values

test_x = sparse.hstack([full_data[full_vars], feature_sparse,st_addr_sparse]).tocsr()[train_size:]
test_y = full_data['target'][train_size:].values


full_vars = full_vars + feature_vars + st_addr_vars
print ("training data size: ", train_x.shape,"testing data size: ", test_x.shape)


params = dict()
params['objective'] = 'multi:softprob'
params['num_class'] = 3
params['eta'] = 0.1
params['max_depth'] = 6
params['min_child_weight'] = 1
params['subsample'] = 0.7
params['colsample_bytree'] = 0.7
params['gamma'] = 1
params['seed']=1234

cv_results = xgb.cv(params, xgb.DMatrix(train_x, label=train_y.reshape(train_x.shape[0],1)),
               num_boost_round=1000000, nfold=5,
       metrics={'mlogloss'},
       seed=1234,
       callbacks=[xgb.callback.early_stop(50)])


training data size:  (49352, 268) testing data size:  (74659, 268)
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[265]	train-mlogloss:0.320483+0.00173725	test-mlogloss:0.529278+0.00641829

Wall time: 4min 19s


In [50]:
%%time
clf = xgb.XGBClassifier(learning_rate = 0.1
                  , n_estimators =246
                  , max_depth = 6
                  , min_child_weight = 1
                  , subsample = 0.7
                  , colsample_bytree = 0.7
                  , gamma = 1
                  , seed = 1234
                  , nthread = -1
                  )




clf.fit(train_x, train_y)

feature_importance = pd.DataFrame(sorted(zip(full_vars,clf.feature_importances_)
                          , key=lambda x: x[1], reverse = True),columns=['feature_name','importance']) 

print (feature_importance.query('importance>0'))

                                      feature_name  importance
0                             hcc_building_id_high    0.034237
1                                price_per_bedroom    0.032046
2                           hcc_building_id_medium    0.031733
3                              hcc_manager_id_high    0.031451
4                                   price_per_room    0.031389
5                                            price    0.031013
6                      price_percentile_by_manager    0.029918
7                    price_percentile_by_disp_addr    0.029104
8                                manager_level_low    0.027477
9                     price_percentile_by_building    0.027258
10                                 price_per_photo    0.027101
11                           hcc_manager_id_medium    0.026569
12                            manager_level_medium    0.025631
13                                    avg_word_len    0.025474
14                              display_address_le    0

In [53]:
feature_importance.to_csv(r'C:\Users\Eric Yang\Desktop\Kaggle Comptetion\Current Competition\Predictions\f.csv')

## Manuanl Tuning

* Greedy-search
* Tune one parameter a time
* The results can be used for further tuning (by Bayesian Optimizer)

In [ ]:
%%time
xgb_scores = pd.DataFrame()
scores = []
for max_depth in [3,4,5,6,7,8,9,10]:

    params = dict()
    params['objective'] = 'multi:softprob'
    params['num_class'] = 3
    params['eta'] = 0.1
    params['max_depth'] = max_depth
    params['min_child_weight'] = 1
    params['subsample'] = 1
    params['colsample_bytree'] = 1
    params['gamma'] = 0
    params['seed']=1234

    cv_results = xgb.cv(params, xgb.DMatrix(train_x, label=train_y.reshape(train_x.shape[0],1)),
                   num_boost_round=1000000,
                   nfold=5,
           metrics={'mlogloss'},
           seed=1234,
           callbacks=[xgb.callback.early_stop(50)])
    best_iteration = len(cv_results)
    best_score = cv_results['test-mlogloss-mean'].min()
    print (max_depth,best_score,best_iteration)
    scores.append([best_score,params['eta'],params['max_depth'],params['min_child_weight'],
                      params['colsample_bytree'],params['subsample'],params['gamma'],best_iteration])
xgb_scores = pd.concat([xgb_scores, pd.DataFrame(scores,columns=['score','eta','max_depth','min_child_weight',
                                   'colsample_bytree','subsample','gamma','best_iteration'])])    
best_max_depth = int(pd.DataFrame(scores,columns=['score','eta','max_depth','min_child_weight',
                                   'colsample_bytree','subsample','gamma','best_iteration']).sort_values(by='score',ascending=True)['max_depth'].values[0])
print ('best max_depth is', best_max_depth)

In [ ]:
%%time
scores = []
for min_child_weight in [1,3,10,30,100]:

    params = dict()
    params['objective'] = 'multi:softprob'
    params['num_class'] = 3
    params['eta'] = 0.1
    params['max_depth'] = best_max_depth
    params['min_child_weight'] = min_child_weight
    params['subsample'] = 1
    params['colsample_bytree'] = 1
    params['gamma'] = 0
    params['seed']=1234

    cv_results = xgb.cv(params, xgb.DMatrix(train_x, label=train_y.reshape(train_x.shape[0],1)),
                   num_boost_round=1000000,
                   nfold=5,
           metrics={'mlogloss'},
           seed=1234,
           callbacks=[xgb.callback.early_stop(50)])
    best_iteration = len(cv_results)
    best_score = cv_results['test-mlogloss-mean'].min()
    print (min_child_weight,best_score,best_iteration)
    scores.append([best_score,params['eta'],params['max_depth'],params['min_child_weight'],
                      params['colsample_bytree'],params['subsample'],params['gamma'],best_iteration])
xgb_scores = pd.concat([xgb_scores, pd.DataFrame(scores,columns=['score','eta','max_depth','min_child_weight',
                                   'colsample_bytree','subsample','gamma','best_iteration'])])    
best_min_child_weight = int(pd.DataFrame(scores,columns=['score','eta','max_depth','min_child_weight',
                                   'colsample_bytree','subsample','gamma','best_iteration']).sort_values(by='score',ascending=True)['min_child_weight'].values[0])
print ('best min_child_weight is', best_min_child_weight)

In [ ]:
%%time
scores = []
for colsample_bytree in [0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]:

    params = dict()
    params['objective'] = 'multi:softprob'
    params['num_class'] = 3
    params['eta'] = 0.1
    params['max_depth'] = best_max_depth
    params['min_child_weight'] = best_min_child_weight
    params['subsample'] = 1
    params['colsample_bytree'] = colsample_bytree
    params['gamma'] = 0
    params['seed']=1234

    cv_results = xgb.cv(params, xgb.DMatrix(train_x, label=train_y.reshape(train_x.shape[0],1)),
                   num_boost_round=1000000,
                   nfold=5,
           metrics={'mlogloss'},
           seed=1234,
           callbacks=[xgb.callback.early_stop(50)])
    best_iteration = len(cv_results)
    best_score = cv_results['test-mlogloss-mean'].min()
    print (colsample_bytree,best_score,best_iteration)
    scores.append([best_score,params['eta'],params['max_depth'],params['min_child_weight'],
                      params['colsample_bytree'],params['subsample'],params['gamma'],best_iteration])
xgb_scores = pd.concat([xgb_scores, pd.DataFrame(scores,columns=['score','eta','max_depth','min_child_weight',
                                   'colsample_bytree','subsample','gamma','best_iteration'])])    
best_colsample_bytree = pd.DataFrame(scores,columns=['score','eta','max_depth','min_child_weight',
                                   'colsample_bytree','subsample','gamma','best_iteration']).sort_values(by='score',ascending=True)['colsample_bytree'].values[0]


print ('best colsample_bytree is', best_colsample_bytree)

In [ ]:
%%time
scores = []
for subsample in [0.6,0.7,0.8,0.9,1]:

    params = dict()
    params['objective'] = 'multi:softprob'
    params['num_class'] = 3
    params['eta'] = 0.1
    params['max_depth'] = best_max_depth
    params['min_child_weight'] = best_min_child_weight
    params['subsample'] = subsample
    params['colsample_bytree'] = best_colsample_bytree
    params['gamma'] = 0
    params['seed']=1234

    cv_results = xgb.cv(params, xgb.DMatrix(train_x, label=train_y.reshape(train_x.shape[0],1)),
                   num_boost_round=1000000,
                   nfold=5,
           metrics={'mlogloss'},
           seed=1234,
           callbacks=[xgb.callback.early_stop(50)])
    best_iteration = len(cv_results)
    best_score = cv_results['test-mlogloss-mean'].min()
    print (subsample,best_score,best_iteration)
    scores.append([best_score,params['eta'],params['max_depth'],params['min_child_weight'],
                      params['colsample_bytree'],params['subsample'],params['gamma'],best_iteration])
xgb_scores = pd.concat([xgb_scores, pd.DataFrame(scores,columns=['score','eta','max_depth','min_child_weight',
                                   'colsample_bytree','subsample','gamma','best_iteration'])])    
best_subsample = pd.DataFrame(scores,columns=['score','eta','max_depth','min_child_weight',
                                   'colsample_bytree','subsample','gamma','best_iteration']).sort_values(by='score',ascending=True)['subsample'].values[0]

print ('best subsample is', best_subsample)

In [ ]:
%%time
scores = []
for gamma in [0,0.5,1,1.5,2]:

    params = dict()
    params['objective'] = 'multi:softprob'
    params['num_class'] = 3
    params['eta'] = 0.1
    params['max_depth'] = best_max_depth
    params['min_child_weight'] = best_min_child_weight
    params['subsample'] = best_subsample
    params['colsample_bytree'] = best_colsample_bytree
    params['gamma'] = gamma
    params['seed']=1234

    cv_results = xgb.cv(params, xgb.DMatrix(train_x, label=train_y.reshape(train_x.shape[0],1)),
                   num_boost_round=1000000,
                   nfold=5,
           metrics={'mlogloss'},
           seed=1234,
           callbacks=[xgb.callback.early_stop(50)])
    best_iteration = len(cv_results)
    best_score = cv_results['test-mlogloss-mean'].min()
    print (gamma,best_score,best_iteration)
    scores.append([best_score,params['eta'],params['max_depth'],params['min_child_weight'],
                      params['colsample_bytree'],params['subsample'],params['gamma'],best_iteration])
xgb_scores = pd.concat([xgb_scores, pd.DataFrame(scores,columns=['score','eta','max_depth','min_child_weight',
                                   'colsample_bytree','subsample','gamma','best_iteration'])])    
best_gamma = pd.DataFrame(scores,columns=['score','eta','max_depth','min_child_weight',
                                   'colsample_bytree','subsample','gamma','best_iteration']).sort_values(by='score',ascending=True)['gamma'].values[0]

print ('best gamma is', best_gamma)

## Automated Tuning

* https://github.com/fmfn/BayesianOptimization


In [54]:
from bayes_opt import BayesianOptimization

xgtrain = xgb.DMatrix(train_x, label=train_y.reshape(train_x.shape[0],1))

def xgb_evaluate(min_child_weight,
                 colsample_bytree,
                 max_depth,
                 subsample,
                 gamma):
    params = dict()
    params['objective'] = 'multi:softprob'
    params['num_class'] = 3
    params['eta'] = 0.1
    params['max_depth'] = int(max_depth )   
    params['min_child_weight'] = int(min_child_weight)
    params['colsample_bytree'] = colsample_bytree
    params['subsample'] = subsample
    params['gamma'] = gamma
    params['verbose_eval'] = True    


    cv_result = xgb.cv(params, xgtrain,
                       num_boost_round=100000,
                       nfold=5,
                       metrics={'mlogloss'},
                       seed=1234,
                       callbacks=[xgb.callback.early_stop(50)])

    return -cv_result['test-mlogloss-mean'].min()


xgb_BO = BayesianOptimization(xgb_evaluate, 
                             {'max_depth': (6, 9),
                              'min_child_weight': (0, 100),
                              'colsample_bytree': (0.1, 0.8),
                              'subsample': (0.9, 1),
                              'gamma': (0, 2)
                             }
                            )

xgb_BO.maximize(init_points=6, n_iter=65)

Initialization
---------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |     gamma |   max_depth |   min_child_weight |   subsample | 
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[169]	train-mlogloss:0.33857+0.0023279	test-mlogloss:0.529338+0.0060731

    1 | 03m35s |   -0.52934 |             0.6283 |    1.2582 |      8.6441 |            19.4144 |      0.8109 | 
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[791]	train-mlogloss:0.415727+0.00195773	test-mlogloss:0.529571+0.00682249

    2 | 04m24s |   -0.52957 |             0.2592 |    1.0328 |      4.0234 |            84.7801 |      0.9292 | 
Multiple eval metrics have been

In [79]:
## Show tuning results
BO_scores = pd.DataFrame(xgb_BO.res['all']['params'])
BO_scores['score'] = pd.DataFrame(xgb_BO.res['all']['values'])
BO_scores = BO_scores.sort_values(by='score',ascending=False)

BO_scores.head()

,colsample_bytree,gamma,max_depth,min_child_weight,subsample,score
41,0.357692,1.917539,7.393037,59.653996,0.988750,-0.527031
35,0.223662,0.054163,8.704360,41.628036,0.989407,-0.527184
56,0.147966,1.981550,6.625051,29.030979,0.995917,-0.527261
61,0.610225,1.984551,9.903677,27.924436,0.960611,-0.527520
40,0.342271,0.035525,9.979960,92.897895,0.993977,-0.527591


### Re-train models

Now we have optimized parameters, let's decrease the size of learning rate and train the model for better results.

Firstly we'll use xgb.cv again to get optimal n_estimators, then we can use tuned n_esimator to finally train the model.

In [57]:
params = dict()
params['objective'] = 'multi:softprob'
params['num_class'] = 3
params['eta'] = 0.01
params['max_depth'] = int(BO_scores.to_dict()['max_depth'][0])
params['min_child_weight'] = BO_scores.to_dict()['min_child_weight'][0]
params['colsample_bytree'] = BO_scores.to_dict()['colsample_bytree'][0]
params['subsample'] = BO_scores.to_dict()['subsample'][0]
params['gamma'] = BO_scores.to_dict()['gamma'][0]
params['seed']=1234

cv_results = xgb.cv(params, xgb.DMatrix(train_x, label=train_y.reshape(train_x.shape[0],1)),
               num_boost_round=1000000, nfold=10,
       metrics={'mlogloss'},
       seed=1234,
       callbacks=[xgb.callback.early_stop(500)])

best_iteration = len(cv_results)


Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 500 rounds.
Stopping. Best iteration:
[4665]	train-mlogloss:0.333136+0.0012015	test-mlogloss:0.521276+0.00964687



In [58]:
clf = xgb.XGBClassifier(n_estimators = best_iteration,
                              learning_rate=0.01,
                              max_depth=int(BO_scores.to_dict()['max_depth'][0]),
                              min_child_weight=BO_scores.to_dict()['min_child_weight'][0],
                              colsample_bytree=BO_scores.to_dict()['colsample_bytree'][0],
                              subsample=BO_scores.to_dict()['subsample'][0],
                              gamma=BO_scores.to_dict()['gamma'][0],
                              seed=1234,
                              nthread=-1)

clf.fit(train_x, train_y)

feature_importance = pd.DataFrame(sorted(zip(full_vars,clf.feature_importances_)
                          , key=lambda x: x[1], reverse = True),columns=['feature_name','importance']) 

print (feature_importance.query('importance>0'))


preds = clf.predict_proba(test_x)
sub_df = pd.DataFrame(preds,columns = ["low", "medium", "high"])
sub_df["listing_id"] = test_data.listing_id.values
sub_df.to_csv(r"C:\Users\Eric Yang\Desktop\Kaggle Comptetion\Current Competition\Predictions\sub_xgb_tuned_v8.csv", index=False)

                                feature_name  importance
0                       hcc_building_id_high    0.031839
1                     hcc_building_id_medium    0.031707
2                          price_per_bedroom    0.030547
3                                      price    0.030518
4                        hcc_manager_id_high    0.030303
5                price_percentile_by_manager    0.030082
6                             price_per_room    0.028758
7                             dist_to_center    0.028652
8              price_percentile_by_disp_addr    0.027419
9                       manager_level_medium    0.026829
10                           price_per_photo    0.025932
11                              avg_word_len    0.025616
12       created_epoch_percentile_by_manager    0.025510
13                     hcc_manager_id_medium    0.025508
14                                  latitude    0.025462
15              price_percentile_by_building    0.025379
16                         mana

In [60]:
interest_levels = ['low', 'medium', 'high']

tau = {
    'low': 0.69195995, 
    'medium': 0.23108864,
    'high': 0.07695141, 
}

def correct(df):
    y = df[interest_levels].mean()
    a = [tau[k] / y[k]  for k in interest_levels]
    print(a)

    def f(p):
        for k in range(len(interest_levels)):
            p[k] *= a[k]
        return p / p.sum()

    df_correct = df.copy()
    df_correct[interest_levels] = df_correct[interest_levels].apply(f, axis=1)

    y = df_correct[interest_levels].mean()
    a = [tau[k] / y[k]  for k in interest_levels]
    print(a)

    return df_correct

In [61]:
sub_df2 = correct(sub_df)
sub_df2.to_csv(r"C:\Users\Eric Yang\Desktop\Kaggle Comptetion\Current Competition\Predictions\sub_xgb_tuned_v8.1.csv", index=False)

[0.99025408244876045, 1.0199119751777115, 1.0307702227597324]
[0.9956691306468165, 1.0076912404379388, 1.0164188334778979]


In [ ]:
def blend_model(clfs, train_x, train_y, test_x, num_class, blend_folds):
    num_class = 3
    blend_folds = 5

    skf = model_selection.StratifiedKFold(n_splits=blend_folds,random_state=1234)
    skf_ids = list(skf.split(train_x, train_y))


    train_blend_x = np.zeros((train_x.shape[0], len(clfs)*num_class))
    test_blend_x = np.zeros((test_x.shape[0], len(clfs)*num_class))
    blend_scores = np.zeros ((blend_folds,len(clfs)))

    print  ("Start blending.")
    for j, clf in enumerate(clfs):
        print ("Blending model",j+1, clf)
        test_blend_x_j = np.zeros((test_x.shape[0], num_class))
        for i, (train_ids, val_ids) in enumerate(skf_ids):
            print ("Model %d fold %d" %(j+1,i+1))
            train_x_fold = train_x[train_ids]
            train_y_fold = train_y[train_ids]
            val_x_fold = train_x[val_ids]
            val_y_fold = train_y[val_ids]
            # Set n_estimators to a large number for early_stopping
            clf.n_estimators = 10000000
            
            # Set evaluation metric
            if type(clf).__name__=='LGBMClassifier':
                metric = 'logloss' #LightGBM
            else:
                metric = 'mlogloss' #XGBoost            
            clf.fit(train_x_fold, train_y_fold,
                    eval_set=[(val_x_fold,val_y_fold)],
                    eval_metric=metric,
                    early_stopping_rounds=500,verbose=False)
            val_y_predict_fold = clf.predict_proba(val_x_fold)
            score = metrics.log_loss(val_y_fold,val_y_predict_fold)
            print ("LOGLOSS: ", score)
            print ("Best Iteration:", clf.best_iteration)
            blend_scores[i,j]=score
            train_blend_x[val_ids, j*num_class:j*num_class+num_class] = val_y_predict_fold
            test_blend_x_j = test_blend_x_j + clf.predict_proba(test_x)
        test_blend_x[:,j*num_class:j*num_class+num_class] = test_blend_x_j/blend_folds
        print ("Score for model %d is %f" % (j+1,np.mean(blend_scores[:,j])))
    return train_blend_x, test_blend_x, blend_scores

In [ ]:
xgb_clfs = []
for p in BO_scores.head(10).iterrows():
    xgb_clfs.append(xgb.XGBClassifier(n_estimators = 10000,
                                  learning_rate=0.01,
                                  max_depth=int(p[1].to_dict()['max_depth']),
                                  min_child_weight=int(p[1].to_dict()['min_child_weight']),
                                  colsample_bytree=p[1].to_dict()['colsample_bytree'],
                                  subsample=p[1].to_dict()['subsample'],
                                  gamma=p[1].to_dict()['gamma'],
                                  seed=1234,
                                  nthread=-1))

In [99]:
knn = KNeighborsClassifier(n_jobs = -1)
knn.fit(train_x, label)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
           weights='uniform')

In [ ]:
from sklearn.linear_model import LogisticRegression,RidgeClassifier
from sklearn.neural_network import MLPClassifier
def search_model(train_x, train_y, est, param_grid, n_jobs, cv, refit=False):
##Grid Search for the best model
    model = model_selection.GridSearchCV(estimator  = est,
                                     param_grid = param_grid,
                                     scoring    = 'log_loss',
                                     verbose    = 10,
                                     n_jobs  = n_jobs,
                                     iid        = True,
                                     refit    = refit,
                                     cv      = cv)
    # Fit Grid Search Model
    model.fit(train_x, train_y)
    print("Best score: %0.3f" % model.best_score_)
    print("Best parameters set:", model.best_params_)
    print("Scores:", model.grid_scores_)
    return model

In [ ]:
param_grid = {"hidden_layer_sizes":[10,50,100,200,500]
              }
model = search_model(train_blend_x_xgb
                                         , train_y
                                         , MLPClassifier()
                                         , param_grid
                                         , n_jobs=-1
                                         , cv=4
                                         , refit=True)   

print ("best subsample:", model.best_params_)




In [ ]:
preds = model.predict_proba(test_blend_x_xgb)
sub_df = pd.DataFrame(preds,columns = ["low", "medium", "high"])
sub_df["listing_id"] = test_data.listing_id.values
sub_df.to_csv(r"C:\Users\Eric Yang\Desktop\Kaggle Comptetion\Current Competition\Predictions/sub_L1xgb_L2MLP_blended_only_v3.csv", index=False)

In [71]:
sub_df2 = correct(sub_df)
sub_df2.to_csv(r"C:\Users\Eric Yang\Desktop\Kaggle Comptetion\Current Competition\Predictions\sub_L1xgb_L2MLP_blended_only_v1.1.csv", index=False)

[1.0023077794348174, 0.99300098174723594, 1.0004625759030992]
[1.0007597967418285, 0.99842529091410981, 0.99791371203792278]
